In [ ]:
import son_analyze
import son_scikit
print('Welcome to son-analyze v{} and son-scikit v{}.'.format(son_analyze.__version__, son_scikit.__version__))

In [ ]:
import helpers
print('You can use and tweak the python code in the helpers.py file (example: "{}")'.format(helpers.foobar()))

In [ ]:
import json
import arrow
import requests
from son_analyze.core.prometheus import PrometheusData
from son_scikit.hl_prometheus import build_sonata_df_by_id

#from http.client import HTTPConnection
#HTTPConnection.debuglevel = 1

query = 'sonemu_rx_count_packets{vnf_name="ubuntu_vnf1",vnf_interface="port1"}'
end = arrow.utcnow()
print('The current date is: {}'.format(end))
start = end.replace(minutes=-50)
payload = {'query': query, 'start': start.timestamp, 'end': end.timestamp, 'step': '1s'}
req = requests.get('http://173.17.0.1:9090/api/v1/query_range?',
                   params=payload)
#print('---')
#print(req.content)

x = PrometheusData(json.dumps(req.json()))
y = build_sonata_df_by_id(x)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

df = y['ubuntu_vnf1']

def sanitize(x):
    return min(15.0, x)

ddf = df.diff().dropna()
ddfn = ddf.copy()
ddfn.sonemu_rx_count_packets = ddfn.sonemu_rx_count_packets.apply(sanitize)
df.plot()
#ddf.plot(logy=True)
ddfn.plot()

In [ ]:
import datetime
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA, ARIMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic


### see http://statsmodels.sourceforge.net/0.6.0/generated/statsmodels.tsa.arima_model.ARMAResults.plot_predict.html

In [ ]:
target_arma = ddfn
results_arma = sm.tsa.ARMA(target_arma, (3, 3)).fit()
print(results_arma.summary2())
fig, ax = plt.subplots()
ax = target_arma.plot(ax=ax, logy=False)
fig = results_arma.plot_predict(start=1, end=len(target_arma.index)+120, ax=ax, plot_insample=False)
plt.show()

In [ ]:
from pyculiarity import detect_ts
import pandas as pd
import time


def f(x):
    dt = x.to_datetime()
    return time.mktime(dt.timetuple())

target = ddf
u = pd.DataFrame({'one': list(target.index.map(f)), 'two': target.sonemu_rx_count_packets})
results = detect_ts(u, max_anoms=0.004, alpha=0.01, direction='both') #, threshold='med_max')

#matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

# make a nice plot
f, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(target.index, target.sonemu_rx_count_packets, 'b')
ax[0].plot(results['anoms'].index, results['anoms']['anoms'], 'ro')
ax[0].set_title('Detected Anomalies')
ax[1].set_xlabel('Time Stamp')
ax[0].set_ylabel('Count')
ax[1].plot(results['anoms'].index, results['anoms']['anoms'], 'b')
ax[1].set_ylabel('Anomaly Magnitude')
plt.show()
    
